In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata,rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("breast_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+---------+-----------+--------+-------------+----------+---------+---------+-------------+--------------+--------------+
|                      | ACURACY | PRECISION | RECALL |     AUC     | F1-SCORE | FP_RATE | FN_RATE | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+---------+-----------+--------+-------------+----------+---------+---------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.95  |    0.95   |  0.95  | 0.991216216 |   0.95   |  0.075  |  0.054  |    0.925    | 0.052631579  | 4.763969158  |
|    DECISION TREE     |   0.95  |    0.95   |  0.95  | 0.947972973 |   0.95   |   0.05  |  0.054  |     0.95    | 0.052631579  | 4.763969158  |
| GAUSSIAN NAIVE BAYES |   0.94  |    0.94   |  0.94  | 0.988851351 |   0.94   |  0.125  |  0.027  |    0.875    | 0.061403509  | 4.763969158  |
|         SVM          |   0.44  |    0.42   |  0.44  | 0.755743243 |   0.43   |   0.85  |  0.4054 |     0.15    | 0.561403509  | 

In [3]:
x_matrix, alt_names = mcdm.load("breast_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weight = [0.10909090908971901,0.16363636363821488, 0.18181818182165294,0.12727272727352062,0.14545454545314052,0.0363636363625124,0.0727272727239339,0.0545454545446777,0.09090909091046283,0.018181818182165292]
weights=np.array(weight)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [4]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


2.0 0.9345384644893263
4.0 0.9047780170462361
3.0 0.923413206070996
7.0 0.24204863508868982
6.0 0.28211079236423886
1.0 0.951446689445082
5.0 0.8712772491802427


In [5]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

2.0 0.028071431747176433
4.0 0.06603815822386921
3.0 0.037148097005403585
6.0 0.9206349206349207
7.0 0.9997732308891574
1.0 0.0
5.0 0.12253710736570203


In [6]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

2.0 0.43939393939590354
3.0 0.3424242424247796
4.0 -0.06060606060836915
7.0 -0.7303030303024877
6.0 -0.6909090909114931
1.0 0.9515151515156199
5.0 -0.2515151515139531


In [7]:
from prettytable import PrettyTable
print("#########################BREAST DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################BREAST DATASET#######################
+----------------------+--------+---------------------+-------+----------------------+-----------+----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |        VALUES        | PROMETHEE |        VALUES        |
+----------------------+--------+---------------------+-------+----------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  2.0   |  0.9345384644893263 |  2.0  | 0.028071431747176433 |    2.0    | 0.43939393939590354  |
|    DECISION TREE     |  4.0   |  0.9047780170462361 |  4.0  | 0.06603815822386921  |    3.0    |  0.3424242424247796  |
| GAUSSIAN NAIVE BAYES |  3.0   |  0.923413206070996  |  3.0  | 0.037148097005403585 |    4.0    | -0.06060606060836915 |
|         SVM          |  7.0   | 0.24204863508868982 |  6.0  |  0.9206349206349207  |    7.0    | -0.7303030303024877  |
|    NEURAL NETWORK    |  6.0   | 0.28211079236423886 |  7.0  |  0.9997732308891574

In [8]:
ans=mcdm.rank(x_matrix, alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1],n_method="Linear2", w_vector=weight, s_method="SAW")  
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
|    RANDOM FOREST     |  0.8000000000030909 |
| LOGISTIC REGRESSION  |  0.776726286558496  |
| GAUSSIAN NAIVE BAYES |  0.7653219129917133 |
|    DECISION TREE     |  0.7545717576572797 |
|     LINEAR MODEL     |  0.7312692939892927 |
|    NEURAL NETWORK    | 0.34697391734782534 |
|         SVM          |  0.3052073810874423 |
+----------------------+---------------------+


In [9]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,n_method="Linear1",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|     LINEAR MODEL     | 0.6465167771584028 |
| LOGISTIC REGRESSION  | 0.5753585017466097 |
|    DECISION TREE     | 0.564548647347012  |
| GAUSSIAN NAIVE BAYES | 0.5597484230340725 |
|         SVM          | 0.4920412475211165 |
|    NEURAL NETWORK    |        0.0         |
|    RANDOM FOREST     |        0.0         |
+----------------------+--------------------+
